In [1]:
import numpy as np

In [47]:
def activation_function(Z):
    if(Z>=0):
        return Z;
    else:
        return 0;

def del_activation_function(Z):
    if(Z>0):
        return 1
    else:
        return 0

def delta_mean(mu,sigma,w_hat):
    return (1/(np.sqrt(2*np.pi)*sigma))*np.exp(-(w_hat-mu)**2/(2*sigma**2))*(w_hat-mu)/sigma**2

def delta_stddev(mu,sigma,w_hat):
    return (np.sqrt(2*np.pi)/(2*np.pi*sigma**2)*np.exp(-(w_hat-mu)**2/(2*sigma**2))*((w_hat-mu)**2/(sigma**2)-1))

def delta_err(y,y_hat,err_type):
    if err_type==0:#mse error
        return (y-y_hat);
    elif err_type==1:#cross entropy error
        return -y/y_hat + (1-y)/(1-y_hat);
    else:
        print("Invalid")

def forward_prop(x1,x2,w1,w2):
    Z = w1*x1+w2*x2
    return (Z,activation_function(Z));

In [58]:
class backprop_prime_with_sgd:
    def __init__(self,epochs):
        self.nepochs = epochs
        self.nlayers = 3
        self.mu = np.array([[1,2],
                  [3,4],
                  [5,6]])
        self.sigma = np.ones((3,2))
        self.weights1 = np.zeros((3,2))
        self.weights2 = np.zeros((3,2))
        self.Z = np.zeros((3,2))
        self.a = np.zeros((3,2))
    
    def backprop(self,y,y_hat,x1,x2):
        #print('Dekh lo:',y,y_hat)
        error = (y - y_hat)**2;
        del_err = delta_err(y,y_hat,0)
        print(self.Z[2,0])
        phi_dash_Z_3_1 = del_activation_function(self.Z[2,0])
        del_phi_dash_Z_2_1 = del_activation_function(self.Z[1,0])
        del_phi_dash_Z_2_2 = del_activation_function(self.Z[1,1])
        del_phi_dash_1_1 = del_activation_function(self.Z[0,0])
        del_phi_dash_Z_1_2 = del_activation_function(self.Z[0,1])
        print(del_err,phi_dash_Z_3_1,del_phi_dash_Z_2_1,del_phi_dash_Z_2_2,del_phi_dash_1_1,del_phi_dash_Z_1_2 )
        #last layer
        del_mu_3_1 = del_err*phi_dash_Z_3_1*self.a[1,0]*delta_mean(self.mu[2,0],self.sigma[2,0],np.mean(self.weights1[2,:]))
        del_sigma_3_1 = del_err*phi_dash_Z_3_1*self.a[1,0]*delta_stddev(self.mu[2,0],self.sigma[2,0],np.mean(self.weights1[2,:]))
        del_mu_3_2 = del_err*phi_dash_Z_3_1*self.a[1,1]*delta_mean(self.mu[2,1],self.sigma[2,1],np.mean(self.weights2[2,:]))
        del_sigma_3_2 = del_err*phi_dash_Z_3_1*self.a[1,1]*delta_stddev(self.mu[2,1],self.sigma[2,1],np.mean(self.weights2[2,:]))
        
        #2nd layer
        del_mu_2_1 = del_err*phi_dash_Z_3_1*self.a[0,0]*delta_mean(self.mu[1,0],self.sigma[1,0],np.mean(self.weights1[1,:]))*(self.weights1[2,0]*del_phi_dash_Z_2_1 + self.weights1[2,1]*del_phi_dash_Z_2_2)
        del_sigma_2_1 = del_err*phi_dash_Z_3_1*self.a[0,0]*delta_stddev(self.mu[1,0],self.sigma[1,0],np.mean(self.weights1[1,:]))*(self.weights1[2,0]*del_phi_dash_Z_2_1 + self.weights1[2,1]*del_phi_dash_Z_2_2)
        del_mu_2_2 = del_err*phi_dash_Z_3_1*self.a[0,1]*delta_mean(self.mu[1,1],self.sigma[1,1],np.mean(self.weights2[1,:]))*(self.weights1[2,0]*del_phi_dash_Z_2_1 + self.weights1[2,1]*del_phi_dash_Z_2_2)
        del_sigma_2_2 = del_err*phi_dash_Z_3_1*self.a[0,1]*delta_mean(self.mu[1,1],self.sigma[1,1],np.mean(self.weights2[1,:]))*(self.weights1[2,0]*del_phi_dash_Z_2_1 + self.weights1[2,1]*del_phi_dash_Z_2_2)
        
        #1st layer
        del_mu_1_1 = del_err*phi_dash_Z_3_1*(self.weights1[2,0]*del_phi_dash_Z_2_1*(self.weights1[1,0]*del_phi_dash_1_1+self.weights2[1,0]*del_phi_dash_Z_1_2)
                                             +self.weights1[2,0]*del_phi_dash_Z_2_2*(self.weights1[1,1]*del_phi_dash_1_1+self.weights2[1,1]*del_phi_dash_Z_1_2))*x1*delta_mean(self.mu[0,0],self.sigma[0,0],np.mean(self.weights1[0,:]))
        del_sigma_1_1 = del_err*phi_dash_Z_3_1*(self.weights1[2,0]*del_phi_dash_Z_2_1*(self.weights1[1,0]*del_phi_dash_1_1+self.weights2[1,0]*del_phi_dash_Z_1_2)
                                             +self.weights1[2,0]*del_phi_dash_Z_2_2*(self.weights1[1,1]*del_phi_dash_1_1+self.weights2[1,1]*del_phi_dash_Z_1_2))*x1*delta_stddev(self.mu[0,0],self.sigma[0,0],np.mean(self.weights1[0,:]))
        del_mu_1_2 = del_err*phi_dash_Z_3_1*(self.weights1[2,0]*del_phi_dash_Z_2_1*(self.weights1[1,0]*del_phi_dash_1_1+self.weights2[1,0]*del_phi_dash_Z_1_2)
                                             +self.weights1[2,0]*del_phi_dash_Z_2_2*(self.weights1[1,1]*del_phi_dash_1_1+self.weights2[1,1]*del_phi_dash_Z_1_2))*x2*delta_mean(self.mu[0,1],self.sigma[0,1],np.mean(self.weights2[0,:]))
        del_sigma_1_2 = del_err*phi_dash_Z_3_1*(self.weights1[2,0]*del_phi_dash_Z_2_1*(self.weights1[1,0]*del_phi_dash_1_1+self.weights2[1,0]*del_phi_dash_Z_1_2)
                                             +self.weights1[2,0]*del_phi_dash_Z_2_2*(self.weights1[1,1]*del_phi_dash_1_1+self.weights2[1,1]*del_phi_dash_Z_1_2))*x2*delta_stddev(self.mu[0,1],self.sigma[0,1],np.mean(self.weights2[0,:]))
        #print(del_mu_1_1,del_sigma_1_1,del_mu_1_2,del_sigma_1_2,del_mu_2_1,del_sigma_2_1,del_mu_2_2,del_sigma_2_2,del_mu_3_1,del_sigma_3_1,del_mu_3_2,del_sigma_3_2)
        return (del_mu_1_1,del_sigma_1_1,del_mu_1_2,del_sigma_1_2,del_mu_2_1,del_sigma_2_1,del_mu_2_2,del_sigma_2_2,del_mu_3_1,del_sigma_3_1,del_mu_3_2,del_sigma_3_2)
    ###############################
    def make_model(self,x1,x2,y):
        a1,a2 = x1,x2
        for layer in range(self.nlayers-1):
            #print('Shapes',self.mu.shape,self.sigma.shape)
            w11,w12 = np.random.normal(self.mu[layer,0],self.sigma[layer,0],2)
            w21,w22 =  np.random.normal(self.mu[layer,1],self.sigma[layer,1],2)
            
            self.weights1[layer,0],self.weights1[layer,1] = w11,w12
            self.weights2[layer,0],self.weights2[layer,1] = w21,w22
            #print("Values sent for forward prop:",a1,a2,w11,w21,self.weights1[0,0],self.weights2[0,0])
            Z1,a1_new = forward_prop(a1,a2,w11,w21)
            #print("weighted_values:",Z1,a1_new)
            self.Z[layer,0] = Z1
            self.a[layer,0] = a1_new
            
            Z2,a2_new = forward_prop(a1,a2,w12,w22)
            
            self.Z[layer,1] = Z2
            self.a[layer,1] = a2_new
            
            a1 = a1_new
            a2 = a2_new
        self.last_weights = np.array([np.random.normal(self.mu[-1,0],self.sigma[-1,0]),np.random.normal(self.mu[-1,1],self.sigma[-1,1])])
        Z3,a3_new = forward_prop(a1,a2,self.last_weights[0],self.last_weights[1])
        self.Z[self.nlayers-1,0] = Z3
        self.a[self.nlayers-1,0] = a3_new
        return Z3,a3_new
    def fit_model(self,X,y):
        for epoch in range(self.nepochs):
            for i in range(len(y)):
                #print('Mu and sigma:',self.mu,self.sigma)
                #print("values of x:",X[i][0],X[i][1])
                Z,a = self.make_model(X[i][0],X[i][1],y[i])
                del_mu_1_1,del_sigma_1_1,del_mu_1_2,del_sigma_1_2,del_mu_2_1,del_sigma_2_1,del_mu_2_2,del_sigma_2_2,del_mu_3_1,del_sigma_3_1,del_mu_3_2,del_sigma_3_2 = self.backprop(y[i],a,X[i][0],X[i][1])
                #print('Returned vals:',del_mu_1_1,del_sigma_1_1,del_mu_1_2,del_sigma_1_2,del_mu_2_1,del_sigma_2_1,del_mu_2_2,del_sigma_2_2,del_mu_3_1,del_sigma_3_1,del_mu_3_2,del_sigma_3_2)
                self.mu[0,0] = self.mu[0,0] - del_mu_1_1
                self.sigma[0,0] = self.sigma[0,0] - del_sigma_1_1
                self.mu[0,1] = self.mu[0,1] - del_mu_1_2
                self.sigma[0,1] = self.sigma[0,1] - del_sigma_2_1
                self.mu[1,0] = self.mu[1,0] - del_mu_2_1
                self.sigma[1,0] = self.sigma[1,0] - del_sigma_2_1
                self.mu[1,1] = self.mu[1,1] - del_mu_2_2
                self.sigma[1,1] = self.sigma[1,1] - del_sigma_2_2
                self.mu[2,0] = self.mu[2,0] - del_mu_3_1
                self.sigma[2,0] = self.sigma[2,0] - del_sigma_3_1
                self.mu[2,1] = self.mu[2,1] - del_mu_3_2
                self.sigma[2,1] = self.sigma[2,1] - del_sigma_3_2
                '''self.sigma = self.sigma - np.array([[del_sigma_1_1,del_sigma_1_2],
                                                   [del_sigma_2_1,del_sigma_2_2],
                                                   [del_sigma_3_1,del_sigma_3_2]])'''
                #print(Z,a)

In [59]:
X = np.array([[0.7,0.3],[0.33,0.67]])
y = np.array([[0],[1]]);
b = backprop_prime_with_sgd(10)
b.fit_model(X,y)

201.843701416506
[-201.84370142] 1 1 1 1 1
102.1785311347613
[-101.17853113] 1 1 1 1 1
40.38391479616756
[-40.3839148] 1 1 1 1 1
56.06637965925897
[-55.06637966] 1 1 1 1 1
22.815484586207067
[-22.81548459] 1 1 1 1 0
-335.50678903516115
[1] 0 1 1 1 1
337.8904133909571
[-337.89041339] 1 1 1 1 1


ValueError: scale < 0

In [53]:
b.weights1

array([[ 0.93550081, -0.62601211],
       [ 3.31166944,  2.74883608],
       [ 0.        ,  0.        ]])

In [54]:
b.weights2

array([[1.89899056, 2.49810863],
       [3.36370588, 3.89234963],
       [0.        , 0.        ]])

In [55]:
b.a

array([[ 1.22454773,  0.31122411],
       [ 5.10216369,  4.57747405],
       [46.93178532,  0.        ]])

In [56]:
b.Z

array([[ 1.22454773,  0.31122411],
       [ 5.10216369,  4.57747405],
       [46.93178532,  0.        ]])

In [57]:
b.last_weights

array([2.22664065, 3.85613376])

In [57]:
forward_prop(X[0][0],X[0][1],b.weights1[0][0],b.weights2[0][0])

(1.1923809912512042, 1.1923809912512042)

In [ ]:
def make_model(self,x1,x2,y):
        a1,a2 = x1,x2
        for layer in range(self.nlayers-1):
            w11,w12 = np.random.normal(self.mu[layer,0],self.sigma[layer,0],2)
            w21,w22 =  np.random.normal(self.mu[layer,1],self.sigma[layer,1],2)
            
            self.weights1[layer,0],self.weights1[layer,1] = w11,w12
            self.weights2[layer,0],self.weights2[layer,1] = w21,w22
            
            Z1,a1_new = forward_prop(a1,a2,w11,w21)
            print("weighted_values:",Z1,a1_new)
            self.Z[layer,0] = Z1
            self.a[layer,0] = a1_new
            
            Z2,a2_new = forward_prop(a1,a2,w12,w22)
            
            self.Z[layer,1] = Z2
            self.a[layer,1] = a2_new
            
            a1 = a1_new
            a2 = a2_new
        self.last_weights = np.array([np.random.normal(self.mu[-1,0],self.sigma[-1,0]),np.random.normal(self.mu[-1,1],self.sigma[-1,1])])
        return forward_prop(a1,a2,self.last_weights[0],self.last_weights[1])